In [ ]:
#IMPORTING NECESSARY LIBRARIES
from pyspark.sql import SparkSession
import mysql.connector
import requests
import pyspark.sql.functions as F
from pyspark.sql import *  
import logging
import os 




: 

In [99]:
#CONFIGURE SPARK SESSION
spark = SparkSession.builder \
    .appName("ReadMySQLTable") \
    .config("spark.jars", "/usr/share/java/mysql-connector-java-9.1.0.jar") \
    .getOrCreate()

In [100]:
# Define context variables
context = {
    "mysql_server": "10.11.12.125",
    "mysql_port": "3306",
    "mysql_database": "PPMWeb_Prod_22_Oct_2024",
    "mysql_user": "teamuser",
    "mysql_password": "sigma5678",
    "import_files_dir": "/home/innv1admn/ppminput/csv/",
    "ImportFilesDir": "D:/MyProjects/Official/ETL/PROJECTOR_PPM/InputFiles/",
    "OutputFilesDir": "D:/MyProjects/Official/ETL/PROJECTOR_PPM/OutputFiles/",
    "ppmdb_Server": "10.11.12.125",
    "ppmdb_Port": "3306",
    "ppmdb_Database": "PPMWeb_Prod_22_Oct_2024",
    "ppmdb_Login": "teamuser",
    "ppmdb_Password": "sigma5678"
}

In [101]:
# Establish MySQL connection
jdbc_url = f"jdbc:mysql://{context['mysql_server']}:{context['mysql_port']}/{context['mysql_database']}"
connection_properties = {
    "user": context["mysql_user"],
    "password": context["mysql_password"],
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [102]:
db_connection = mysql.connector.connect(
    host=context["ppmdb_Server"],
    port=context["ppmdb_Port"],
    database=context["ppmdb_Database"],
    user=context["ppmdb_Login"],
    password=context["ppmdb_Password"]
)


In [105]:
# Fetch CSV File from URL
url = "https://app.projectorpsa.com/report/code/ProjectorPPMAccountList?format=csv&columnheaders=firstrow&UserName=bala.peddi@prolifics.com&Password=Mylife@2024"
response = requests.get(url)
csv_file_path = f"{context['import_files_dir']}/projectorAccounts.csv"
with open(csv_file_path, 'wb') as file:
    file.write(response.content)

<span style="color:red"> <b>changes made in below cell<b> </span><br>
<span style="color: green"> column names are renamed by removing the space between words like below</span><br>
<span style="color:blue"> Project Location ---->ProjectLocation</span><br>
<span style="color:blue"> Client Industry---->ClientIndustry</span><br>
<span style="color:blue"> Client SFA URL---->ClientSFAURL</span><br>
<span style="color:blue"> Client Inactive (Yes/No) ---->ClientInactive</span><br>
<span style="color:blue"> Client Number ---->ClientNumber</span><br>
<span style="color:blue"> Client Name ---->ClientName</span><br>


In [106]:

df =  (spark.read.csv(csv_file_path, header=True, inferSchema=True)
        .withColumnRenamed("Project Location","ProjectLocation") # filed name are renamed to ProjectLocation
        .withColumnRenamed("Client Industry","ClientIndustry") # filed name are renamed to ClientIndustry
        .withColumnRenamed("Client SFA URL","ClientSFAURL") # filed name are renamed to ClientSFAURL
        .withColumnRenamed("Client Inactive (Yes/No)","ClientInactive") # filed name are renamed to ClientInactive
        .withColumnRenamed("Client Number","ClientNumber")# filed name are renamed to ClientNumber
        .withColumnRenamed("Client Name","ClientName"))# filed name are renamed to ClientName


In [66]:
df.show()

+-------------+--------------------+--------------------+--------------------+--------------+---------------+
| ClientNumber|          ClientName|      ClientIndustry|        ClientSFAURL|ClientInactive|ProjectLocation|
+-------------+--------------------+--------------------+--------------------+--------------+---------------+
|       170S01|         170 Systems|             Finance|                NULL|           Yes|             US|
|1ST FORMATION|1st Formations Li...|               Other|                 tbd|            No|             UK|
|41STPARMTER01|The 41st Paramete...|   Computer Services|                NULL|           Yes|             US|
|       AAMC01|Association of Am...|           Education|                NULL|           Yes|             US|
|        ABC01|          ABC Disney|         Media & Ent|                NULL|           Yes|             US|
|       ACBL01|American Contract...|         Media & Ent|                NULL|           Yes|             US|
|    ACCEN

<div style="text-align: center;"><span style="color:GREEN"> <b>------------- PART 1-------------<b> </span><br></div>


In [107]:
sales_territory_df = df.select(
            F.expr("substring(ProjectLocation, 1, 9)").alias("territory_code"), 
            F.regexp_replace("ProjectLocation", "[^\\x00-\\x7F]", "").alias("full_name"),
            F.regexp_replace("ProjectLocation", "[^\\x00-\\x7F]", "").alias("territory_location"),
            F.regexp_replace("ProjectLocation", "[^\\x00-\\x7F]", "").alias("description")
        )

In [108]:
# Reading data from Sales Territory table 

existing_territories_df = spark.read.jdbc(jdbc_url, "sales_territories", properties=connection_properties)


In [109]:
# Performing lookup 

new_territories_df = sales_territory_df.join(existing_territories_df, sales_territory_df.territory_code == existing_territories_df.territory_code, "left_anti")


In [70]:
new_territories_df.show() 

+--------------+---------+------------------+-----------+
|territory_code|full_name|territory_location|description|
+--------------+---------+------------------+-----------+
+--------------+---------+------------------+-----------+



In [260]:
#WRITING THE FILTERED DATA BACK TO sales_territories TABLE

# new_territories_df.write.jdbc(jdbc_url, "sales_territories", mode="append", properties=connection_properties)

<div style="text-align: center;"><span style="color:GREEN"> <b>------------- PART 2-------------<b> </span><br></div>


<span style="color:red"> <b>Changes made in below cell<b></span><br>
<span style="color: green">Modified like below</span><br>
<span style="color:blue">selectExpr -----------> select</span>



In [73]:
# Transform Industry Data 

industry_df = df.select(        
    F.lit(2).alias("lkup_type_group_id"),
    F.col("ClientIndustry").alias("lkup_name"),
    F.col("ClientIndustry").alias("description"),
    F.lit(True).alias("is_active")
) 

In [74]:
# READ THE DATA FROM lkup_types TABLE FROM MYSQL

existing_industries_df = spark.read.jdbc(jdbc_url, "lkup_types", properties=connection_properties).filter("lkup_type_group_id = 2")

In [75]:
#PERFORM LOOKUP AND FILTER OUT THE DATA

new_industries_df = industry_df.join(existing_industries_df, industry_df.lkup_name == existing_industries_df.lkup_name, "left_anti")


In [76]:
new_industries_df.show() 

+------------------+---------+-----------+---------+
|lkup_type_group_id|lkup_name|description|is_active|
+------------------+---------+-----------+---------+
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|     true|
|                 2|     NULL|       NULL|    

In [ ]:
#WRITE FILTERED DATA BACK TO lkup_types TABLE

# new_industries_df.write.jdbc(jdbc_url, "lkup_types", mode="append", properties=connection_properties)

<div style="text-align: center;"><span style="color:BLACK"> <b>--------------------- PART 3-----------------<b> </span><br></div>


<span style="color:RED"><b> BELOW CODE BLOCK IS NEWLY ADDED<b> (Not present in converted code) </span><br>
<span style="color:green">This code is added to perform lookup among sales_territories table, lkup_types with input df</span><br>

<span style="color:blue">This code block is responsible to perform lookup (according to talend job)</span>

In [82]:
#performing lookup among sales_territories,lkup_types with input df

df_final = df.join(
    existing_territories_df,
    df["ProjectLocation"] == existing_territories_df["territory_code"],
    "left_outer" 
).withColumnRenamed("id","sales_territ_id").join(
    existing_industries_df,
    df["ClientIndustry"] == existing_industries_df["lkup_name"],
    "left_outer" )



<span style="color:RED"><b> CHANGES MADE IN BELOW CELL </span><br>
<span style="color:green">Added below lines of code to map typ_industry_id,sales_territory_id </span><br>
<span style="color:blue"><b>1.</b> F.col("id").alias("typ_industry_id")----> This id is coming from lkup_table table </span><br>
<span style="color:blue"><b>2.</b> F.col("sales_territ_id").alias("sales_territory_id")----> This sales_territ_id is coming from sales_territories table </span><br>





In [84]:
#mapping the columns from after performing look up
client_df = df_final.select( 
    F.col("ClientNumber").alias("cust_code"),
    F.regexp_replace("ClientName", "[^\\x00-\\x7F]", "").alias("full_name"),
    F.col("id").alias("typ_industry_id"),
    F.col("ClientSFAURL").alias("website"),
    F.when(F.col("ClientInactive") == "No", 161).otherwise(162).alias("typ_cust_status_id"),
    F.col("sales_territ_id").alias("sales_territory_id"),
    F.when(F.col("ClientInactive") == "No", True).otherwise(False).alias("is_active")   
)


In [86]:
#AGGREGATE DATA BY cust_code, full_name FILEDS 

aggregated_client_df = (client_df.groupBy("cust_code", "full_name").agg(
    F.max("typ_industry_id").alias("typ_industry_id"), 
    F.max("website").alias("website"), 
    F.max("typ_cust_status_id").alias("typ_cust_status_id"), 
    F.max("sales_territory_id").alias("sales_territory_id"), 
    F.first("is_active").alias("is_active"), 
))


In [87]:
aggregated_client_df.show()

+-------------+--------------------+---------------+--------------------+------------------+------------------+---------+
|    cust_code|           full_name|typ_industry_id|             website|typ_cust_status_id|sales_territory_id|is_active|
+-------------+--------------------+---------------+--------------------+------------------+------------------+---------+
|       170S01|         170 Systems|            206|                NULL|               162|                47|    false|
|1ST FORMATION|1st Formations Li...|            214|                 tbd|               161|                52|     true|
|41STPARMTER01|The 41st Paramete...|            203|                NULL|               162|                47|    false|
|       AAMC01|Association of Am...|            204|                NULL|               162|                47|    false|
|        ABC01|          ABC Disney|            545|                NULL|               162|                47|    false|
|       ACBL01|American 

In [ ]:
# WRITING AGGREGATED DATA TO projector_customer_tmp TABLE 

# aggregated_client_df.write.jdbc(jdbc_url, "projector_customer_tmp", mode="overwrite", properties=connection_properties)


<div style="text-align: center;"><span style="color:BLACK"> <b>--------CALLING ProcessProjectorAccounts STORED PROCEDURE--------<b> </span><br></div>


In [88]:
conn = mysql.connector.connect(
    host=context['mysql_server'],
    user=context['mysql_user'],
    password=context['mysql_password'],
    database=context['mysql_database']
) 

cursor = conn.cursor()
cursor.callproc('ProcessProjectorAccounts')
conn.commit()

#--------------------------UNCOMMENT BELOW SNIPPET IF WANT TO SEE DATA RETURNED BY PROCEDURE-----------------

# first_result = True
# for result in cursor.stored_results():
#     if first_result:
        
#         rows = result.fetchall()
#         columns = [desc[0] for desc in result.description]  

#         df_ProcessProjectorAccounts_proc = spark.createDataFrame(rows, schema=columns)
#         df_ProcessProjectorAccounts_proc.show()
#         first_result = False 
#     else:
#         break  

cursor.close()
conn.close()


<div style="text-align: center;"><span style="color:BLACK"> <b>--------CALLING SyncCustomersWithProspects STORED PROCEDURE--------<b> </span><br></div>


In [89]:
conn = mysql.connector.connect(
    host=context['mysql_server'],
    user=context['mysql_user'],
    password=context['mysql_password'],
    database=context['mysql_database']
) 

cursor = conn.cursor()
cursor.callproc('SyncCustomersWithProspects')
conn.commit()

#--------------------------UNCOMMENT BELOW SNIPPET IF WANT TO SEE DATA RETURNED BY PROCEDURE-------------------

# first_result = True
# for result in cursor.stored_results():
#     if first_result:
        
#         rows = result.fetchall()
#         columns = [desc[0] for desc in result.description]  

#         df_SyncCustomersWithProspects_proc = spark.createDataFrame(rows, schema=columns)
#         df_SyncCustomersWithProspects_proc.show()
#         first_result = False 
#     else:
#         break  


cursor.close()
conn.close()


<div style="text-align: center;"><span style="color:BLACK"> <b>----------------------PART 4-----------------------<b> </span><br></div>


In [90]:

customer_division_df = df.groupBy("ClientNumber").agg(
    F.first("ClientName").alias("ClientName"),
    F.first("ClientIndustry").alias("ClientIndustry"),
    F.first("ClientSFAURL").alias("ClientSFAURL"),
    F.first("ClientInactive").alias("ClientInactive"),
    F.first("ProjectLocation").alias("ProjectLocation")
)


<span style="color:RED"><b> CHANGES MADE IN BELOW CELL </span><br>
<span style="color:green">Removed below lines from the code as they are wrongly mapped and not required according to talend job</span><br>
<span style="color:blue">F.regexp_replace("ClientName", "[^\\x00-\\x7F]", "").alias("division_name"),</span><br>
<span style="color:blue">F.regexp_replace("ClientIndustry", "[^\\x00-\\x7F]", "").alias("industry"), </span><br><span style="color:blue">F.regexp_replace("ClientSFAURL", "[^\\x00-\\x7F]", "").alias("website"),</span><br><span style="color:blue">F.regexp_replace("ProjectLocation", "[^\\x00-\\x7F]", "").alias("location")</span><br>





In [91]:
# changes made (commented last 4 columns as we dont require them and those are not [resent in talend mapping)
copyOfcustDivs = customer_division_df.select(
    F.lit(None).alias("id"),
    F.lit("Customer").alias("object_type_code"),
    F.when(F.col("ClientInactive") == "No", False).otherwise(True).alias("is_active"),
    F.lit(None).alias("date_created"),
    F.lit(None).alias("last_updated"),
    F.lit("").alias("created_by"),
    F.lit("").alias("last_updated_by"),
    F.lit(1).alias("created_by_id"),
    F.lit(1).alias("last_updated_by_id"),
    F.lit("UNKNOWN").alias("DivisionName"),
    F.lit("UNKNOWN").alias("DivisionDescription"),
    F.lit("UNKNOWN").alias("DivisionAddress"),
    F.lit("UNKNOWN").alias("DivisionContact"),
    F.col("ClientNumber").alias("CustomerId")
    # F.regexp_replace("ClientName", "[^\\x00-\\x7F]", "").alias("division_name"),
    # F.regexp_replace("ClientIndustry", "[^\\x00-\\x7F]", "").alias("industry"),
    # F.regexp_replace("ClientSFAURL", "[^\\x00-\\x7F]", "").alias("website"),
    # F.regexp_replace("ProjectLocation", "[^\\x00-\\x7F]", "").alias("location")
)

In [ ]:
#----------------REST STEPS ARE MISSING AFTER THIS IN  FLIT 2580------------------------------

<span style="color:RED"><b> CHANGES MADE IN BELOW CELL </span><br>
<span style="color:green">Code in below 4 cells are taken from flit_2581 </span><br>
<span style="color:blue">Next steps after mapping are not present in flit_2580, so the  script required for part 4 is taken from  flit_2581 </span><br>






In [93]:
customer_division_df = df.select("ClientNumber", "ClientName", "ClientIndustry", "ClientSFAURL", "ClientInactive", "ProjectLocation")

# Aggregate Customer Division Data
aggregated_customer_division_df = customer_division_df.groupBy("ClientNumber").agg(
    F.first("ClientName").alias("ClientName"),
    F.first("ClientIndustry").alias("ClientIndustry"),
    F.first("ClientSFAURL").alias("ClientSFAURL"),
    F.first("ClientInactive").alias("ClientInactive"),
    F.first("ProjectLocation").alias("ProjectLocation")
)
aggregated_customer_division_df.show()


+-------------+--------------------+--------------------+--------------------+--------------+---------------+
| ClientNumber|          ClientName|      ClientIndustry|        ClientSFAURL|ClientInactive|ProjectLocation|
+-------------+--------------------+--------------------+--------------------+--------------+---------------+
|       170S01|         170 Systems|             Finance|                NULL|           Yes|             US|
|1ST FORMATION|1st Formations Li...|               Other|                 tbd|            No|             UK|
|41STPARMTER01|The 41st Paramete...|   Computer Services|                NULL|           Yes|             US|
|       AAMC01|Association of Am...|           Education|                NULL|           Yes|             US|
|        ABC01|          ABC Disney|         Media & Ent|                NULL|           Yes|             US|
|       ACBL01|American Contract...|         Media & Ent|                NULL|           Yes|             US|
|    ACCEN

In [110]:
# Transform Customer Division Data

customer_division_transformed_df = aggregated_customer_division_df.select(
    F.col("ClientName").alias("DivisionName"),
    F.col("ClientNumber").alias("ClientNumber"),
    F.col("ClientIndustry").alias("DivisionDescription"),
    F.col("ClientSFAURL").alias("DivisionAddress"),
    F.col("ClientInactive").alias("DivisionContact")
)

In [112]:
#RAED CUSTOMERS ATBLE FROM MYSQL
customer_lookup_df = spark.read.jdbc(jdbc_url, "customers", properties=connection_properties) \
    .select("id", "cust_code")


In [114]:
#PERFORM LOOKUP 
customer_division_transformed_df = customer_division_transformed_df \
    .join(customer_lookup_df, customer_division_transformed_df.ClientNumber == customer_lookup_df.cust_code, "left") \
    .select("DivisionName", "DivisionDescription", "DivisionAddress", "DivisionContact", F.col("id").alias("CustomerId")
           )

In [115]:
customer_division_transformed_df.show()

+--------------------+--------------------+--------------------+---------------+----------+
|        DivisionName| DivisionDescription|     DivisionAddress|DivisionContact|CustomerId|
+--------------------+--------------------+--------------------+---------------+----------+
|         170 Systems|             Finance|                NULL|            Yes|   1551272|
|1st Formations Li...|               Other|                 tbd|             No| 128283684|
|The 41st Paramete...|   Computer Services|                NULL|            Yes|   1551273|
|Association of Am...|           Education|                NULL|            Yes|   1551274|
|          ABC Disney|         Media & Ent|                NULL|            Yes|   1551275|
|American Contract...|         Media & Ent|                NULL|            Yes|   1551276|
|Accenture (UKDIRECT)|   Computer Services|                NULL|            Yes|   1551277|
|  Accenture (UKISSI)|   Computer Services|                NULL|            Yes|

In [ ]:
# Insert Customer Divisions into Base Domains

# customer_division_transformed_df.write.jdbc(mysql_url, "base_domains", mode="append", properties=mysql_properties)


<span style="color:RED"><b> CHANGES MADE IN BELOW CELL </span><br>
<span style="color:green">Replacing the existing code the below line to hardcode last_inserted_id as <b>'999'</b></span><br>
<span style="color:blue">As we are not sure about LAST_INSERT_ID(), we are considering last_insert_id as 999 temporarily </span><br>
<span style="color:black">last_insert_id_df = spark.sql("SELECT LAST_INSERT_ID() as last_insert_id")<b>----></b> last_insert_id_df = spark.createDataFrame([Row(id='999')]) </span><br>





In [120]:
# Get Last Insert ID 

# last_insert_id_df = spark.sql("SELECT LAST_INSERT_ID() as last_insert_id")

last_insert_id_df = spark.createDataFrame([Row(id='999')])

In [121]:
# Map Last Insert ID to Division Data

division_data_df = customer_division_transformed_df.crossJoin(last_insert_id_df)


In [123]:
division_data_df.show()

+--------------------+--------------------+--------------------+---------------+----------+---+
|        DivisionName| DivisionDescription|     DivisionAddress|DivisionContact|CustomerId| id|
+--------------------+--------------------+--------------------+---------------+----------+---+
|         170 Systems|             Finance|                NULL|            Yes|   1551272|999|
|1st Formations Li...|               Other|                 tbd|             No| 128283684|999|
|The 41st Paramete...|   Computer Services|                NULL|            Yes|   1551273|999|
|Association of Am...|           Education|                NULL|            Yes|   1551274|999|
|          ABC Disney|         Media & Ent|                NULL|            Yes|   1551275|999|
|American Contract...|         Media & Ent|                NULL|            Yes|   1551276|999|
|Accenture (UKDIRECT)|   Computer Services|                NULL|            Yes|   1551277|999|
|  Accenture (UKISSI)|   Computer Servic

In [ ]:
# Insert or Update Divisions


# division_data_df.write.jdbc(mysql_url, "divisions", mode="append", properties=mysql_properties)